In [1]:
pip install pandas scikit-learn nltk flask

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv("Symptom2Disease_Final.csv")

# Explore the dataset
df = df.dropna()# Ensure columns are correctly named
df.columns = df.columns.str.strip() 



In [3]:
print(df.info())  # Check for missing values and data types


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   disease                  1209 non-null   object
 1   symptoms                 1209 non-null   object
 2   age                      1209 non-null   object
 3   gender                   1209 non-null   object
 4   severity                 1209 non-null   object
 5   duration                 1209 non-null   object
 6   lifestyle                1209 non-null   object
 7   family_history           1209 non-null   object
 8   pre_existing_conditions  1209 non-null   object
 9   medications              1209 non-null   object
 10  recommended_tests        1209 non-null   object
dtypes: object(11)
memory usage: 104.0+ KB
None


In [4]:
print(df['label'].value_counts())  # Check the distribution of diseases

KeyError: 'label'

In [ ]:
#Step 3: Preprocess the Data
Malaria = df[df['label'] == 'Malaria']
print(Malaria.head())

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [14]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
print(nltk.data.path)
nltk.data.path.append('C:/Users/ASUS/nltk_data')  # Adjust to your path


['C:\\Users\\ASUS/nltk_data', 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data', 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data', 'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data', 'C:\\Users\\ASUS\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# Text cleaning function
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    return ' '.join(tokens)

# Apply cleaning to the dataset
df['cleaned_symptoms'] = df['symptoms'].apply(clean_text)

In [ ]:
# Apply cleaning to the dataset
df['cleaned_text'] = df['text'].apply(clean_text)

In [ ]:
# Check a few examples of cleaned symptoms
print(df[['symptoms', 'cleaned_symptoms']].head(10))


                                            symptoms  \
0  I have been experiencing a skin rash on my arm...   
1  My skin has been peeling, especially on my kne...   
2  I have been experiencing joint pain in my fing...   
3  There is a silver like dusting on my skin, esp...   
4  My nails have small dents or pits in them, and...   
5  The skin on my palms and soles is thickened an...   
6  The skin around my mouth, nose, and eyes is re...   
7  My skin is very sensitive and reacts easily to...   
8  I have noticed a sudden peeling of skin at dif...   
9  The skin on my genitals is red and inflamed. I...   

                                    cleaned_symptoms  
0  experiencing skin rash arms legs torso past we...  
1  skin peeling especially knees elbows scalp pee...  
2  experiencing joint pain fingers wrists knees p...  
3  silver like dusting skin especially lower back...  
4  nails small dents pits often feel inflammatory...  
5  skin palms soles thickened deep cracks cracks ... 

In [ ]:
#Step 4: Split the Data into Training and Testing Sets

In [8]:
from sklearn.model_selection import train_test_split

# Split the data (Use "cleaned_symptoms" and "disease" instead of "text" and "label")
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_symptoms'], df['disease'], test_size=0.2, random_state=42)


print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))


Training set size: 967
Testing set size: 242


In [ ]:
#Step 5: Vectorize the Text Data

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


print("Shape of training data:", X_train_vec.shape)
print("Shape of testing data:", X_test_vec.shape)

Shape of training data: (967, 1387)
Shape of testing data: (242, 1387)


In [ ]:
# Train a Machine Learning Model

In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the model
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# Make predictions
y_pred = model.predict(X_test_vec)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9214876033057852
Classification Report:
                                  precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00        11
                      Arthritis       1.00      1.00      1.00         9
               Bronchial Asthma       0.73      1.00      0.84         8
           Cervical spondylosis       0.88      1.00      0.93         7
                    Chicken pox       1.00      0.85      0.92        13
                    Common Cold       0.92      0.92      0.92        12
                         Dengue       0.92      0.85      0.88        13
          Dimorphic Hemorrhoids       1.00      1.00      1.00        11
               Fungal infection       1.00      1.00      1.00        10
                   Hypertension       1.00      1.00      1.00         4
                       Impetigo       1.00      1.00      1.00        11
                       Jaundice       1.00      1.00      1.00        

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test_vec)

# Get the confidence scores (probability)
y_pred_proba = model.predict_proba(X_test_vec)

# Convert probabilities to max confidence scores
confidence_scores = y_pred_proba.max(axis=1)

# Show some sample predictions
for i in range(5):  # Print first 5 predictions
    print(f"Symptoms: {X_test.iloc[i]}")
    print(f"Predicted Disease: {y_pred[i]}")
    print(f"Confidence Score: {round(confidence_scores[i], 2)}")
    print("-" * 50)


In [ ]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
#Step 7: Save the Model and Vectorizer

In [11]:
import joblib

# Save the trained model
joblib.dump(model, 'symptom_checker_model.pkl')

# Save the TF-IDF vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [ ]:
!pip install flask joblib

In [12]:
import joblib

# Load the model and vectorizer
model = joblib.load('symptom_checker_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

In [ ]:
from flask import Flask, request, jsonify
import joblib
import threading

In [ ]:
# Initialize the Flask app
app = Flask(__name__)

In [ ]:
# Text cleaning function
def clean_text(text):
    import re
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize

    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    return ' '.join(tokens)

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    user_input = request.json['symptoms']

    # Clean and process input
    cleaned_input = clean_text(user_input)
    input_vec = vectorizer.transform([cleaned_input])

    # Get prediction and confidence
    prediction_prob = model.predict_proba(input_vec)
    predicted_label = model.classes_[prediction_prob.argmax()]
    confidence = round(prediction_prob.max(), 2)

    # Get additional details
    disease_info = df[df['disease'] == predicted_label].iloc[0]
    recommended_tests = eval(disease_info['recommended_tests'])  # Convert string to list
    medications = eval(disease_info['medications'])  # Convert string to list

    return jsonify({
        "predicted_disease": predicted_label,
        "confidence": confidence,
        "recommended_tests": recommended_tests,
        "medications": medications
    })


In [ ]:
import requests

# Define the API endpoint
url = 'http://127.0.0.1:5000/predict'

# Define the input data
data = {
    'symptoms': 'There is bruising on my legs that I cannot explain. I can see strange blood vessels below the skin.'
}

# Send a POST request to the API
response = requests.post(url, json=data)

# Print the response
print(response.json())